In [ ]:
# Third-party
import pandas as pd
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline

from scipy.spatial import Delaunay

In [ ]:
sgr = np.genfromtxt("/Users/adrian/projects/streams/data/simulation/LM10/SgrTriax_DYN.dat", 
                    delimiter=" ", dtype=None, names=True)

In [ ]:
sgr_c = coord.SkyCoord(ra=sgr['ra']*u.degree, 
                       dec=sgr['dec']*u.degree,
                       distance=sgr['dist']*u.kpc)

In [ ]:
hsc_gama = pd.read_csv("/Users/adrian/Downloads/gama15RaDec.txt", sep=" ", names=['ra', 'dec'], skiprows=1)
hsc_xmm = pd.read_csv("/Users/adrian/Downloads/xmmRaDec.txt", sep=" ", names=['ra', 'dec'], skiprows=1)

hsc_fields = {'gama': hsc_gama, 'xmm': hsc_xmm}

In [ ]:
hsc_c = {name: coord.SkyCoord(ra=np.asarray(f['ra'])*u.deg, dec=np.asarray(f['dec'])*u.deg)
         for name,f in hsc_fields.items()}

Note: this is wrong because the tesselation is done on angular coordinates...

In [ ]:
hsc_hulls = {name: Delaunay(np.vstack((c.ra.degree,c.dec.degree)).T) for name,c in hsc_c.items()}

In [ ]:
sgr_pts = np.vstack((sgr_c.ra.degree, sgr_c.dec.degree)).T

sgr_in_hsc_idx = {}
for name,hull in hsc_hulls.items():
    sgr_in_hsc_idx[name] = hull.find_simplex(sgr_pts) >= 0

---

## Plot all Sgr model stars colored by distance

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(15,8),subplot_kw=dict(projection='aitoff'))

cb = ax.scatter(sgr_c.ra.wrap_at(180*u.degree).radian, sgr_c.dec.radian, 
               c=sgr_c.distance.kpc, cmap='magma', vmin=3, vmax=60, s=2)

for c in hsc_c.values():
    ax.plot(c.ra.wrap_at(180*u.degree).radian[::1000], c.dec.radian[::1000], 
            linestyle='none', marker=',', color='g')

cbar = fig.colorbar(cb)
cbar.set_label('Dist. [kpc]')

## Plot only Sgr stars in convex hull of HSC fields

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(15,8),subplot_kw=dict(projection='aitoff'))

for name,idx in sgr_in_hsc_idx.items():
    cb = ax.scatter(sgr_c[idx].ra.wrap_at(180*u.degree).radian, sgr_c[idx].dec.radian, 
                    c=sgr_c[idx].distance.kpc, cmap='magma', vmin=3, vmax=60, s=2)
    
cbar = fig.colorbar(cb)
cbar.set_label('Dist. [kpc]')

## Distance / distance modulus histograms for the two fields

In [ ]:
dist_bins = np.linspace(0.,80.,16) # kpc
distmod_bins = np.linspace(14.5,20.,16) # kpc

for name,idx in sgr_in_hsc_idx.items():
    fig,axes = pl.subplots(1,2,figsize=(10,5))
    axes[0].hist(sgr_c.distance.kpc[idx], bins=dist_bins)
    axes[1].hist(sgr_c.distance.distmod[idx], bins=distmod_bins)
    
    axes[0].set_xlabel("Distance [kpc]")
    axes[1].set_xlabel("Distance modulus [mag]")
    
    fig.suptitle("Field: {}".format(name), fontsize=24, y=1.0)
    fig.tight_layout()
    fig.subplots_adjust(top=0.9)